In [1]:
import os,cv2
import numpy as np
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras import backend as K

from keras.utils import np_utils
from keras import optimizers
from keras.models import Sequential
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from keras.models import model_from_json, load_model

Using TensorFlow backend.


In [2]:
K.clear_session() # Clear previous models from memory.

json_file = open('bacteria_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("bacteria-classifier.h5")
#loaded_model=load_model('bacteria-classifier.h5')

loaded_model.summary()














_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv1 (Conv2D)               (None, 32, 242, 242)      320       
_________________________________________________________________
M1 (MaxPooling2D)            (None, 32, 121, 121)      0         
_________________________________________________________________
Conv2 (Conv2D)               (None, 64, 119, 119)      18496     
_________________________________________________________________
M2 (MaxPooling2D)            (None, 64, 59, 59)        0         
_________________________________________________________________
Flatten (Flatten)            (None, 222784)            0         
_________________________________________________________________
Dense1 (Dense)               (None, 64)                14258240  
_________________________________________________________________
Dense2 (Dense)               (None, 4)                 260      

In [3]:
PATH = os.getcwd()
data_path = PATH + '/dataset'
data_dir_list = os.listdir(data_path)
print(data_dir_list)
va_labels_name={'Candida_albicans':0,'Clostridium_perfringens':1,'Lactobacillus_paracasei':2,'Staphylococcus_saprophiticus':3}

validation_image_list=[]
validation_label_list=[]

validation_data_path = PATH + '/dataset/Validation'
validation_data_dir_list = os.listdir(validation_data_path)
print(validation_data_path)

for folders in validation_data_dir_list:
    va_img_list=os.listdir(validation_data_path+'/'+ folders)
    print ('Loading the images of dataset-'+'{}\n'.format(folders))
    label = va_labels_name[folders]

    for img in va_img_list:
        input_img=cv2.imread(validation_data_path + '/'+ folders + '/'+ img )
        input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
        validation_image_list.append(input_img)
        validation_label_list.append(label)

['Train', 'Validation']
C:\Users\rohan\Documents\Project - Deep Learning/dataset/Validation
Loading the images of dataset-Candida_albicans

Loading the images of dataset-Clostridium_perfringens

Loading the images of dataset-Lactobacillus_paracasei

Loading the images of dataset-Staphylococcus_saprophiticus



In [4]:
va_Images = np.array(validation_image_list)
va_Images = va_Images.astype('float32')
va_Labels = np.array(validation_label_list)
print(np.unique(va_Labels))
va_Labels = np_utils.to_categorical(va_Labels)
print(va_Labels.shape)
print(va_Labels)
va_Images = va_Images.reshape(va_Images.shape[0],1, 244, 244).astype('float32')
## converting the pixel values in terms of zero and ones using min max scaling
va_Images = (va_Images-np.min(va_Images))/(np.max(va_Images)-np.min(va_Images))
print(va_Images.shape)

testPred = loaded_model.predict_classes(va_Images)
predProbabilities = loaded_model.predict(va_Images)

[0 1 2 3]
(1438, 4)
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]
(1438, 1, 244, 244)


In [5]:
np.savetxt("bacteria_pred.csv", testPred, fmt="%s", delimiter=" ", newline="\n")
np.savetxt("bacteria_pred_probabilities.csv", predProbabilities, fmt="%s", delimiter=" ", newline="\n")

testLabelsOriginal = np.argmax(va_Labels, axis=1)

print("Classification report ")
print(classification_report(y_true=testLabelsOriginal, y_pred=testPred))

Classification report 
              precision    recall  f1-score   support

           0       0.98      0.92      0.95       426
           1       0.85      0.78      0.81       282
           2       0.98      1.00      0.99       377
           3       0.85      0.95      0.90       353

    accuracy                           0.92      1438
   macro avg       0.92      0.91      0.91      1438
weighted avg       0.92      0.92      0.92      1438



In [6]:
print("Confusion matrix ")
print(confusion_matrix(y_true=testLabelsOriginal, y_pred=testPred))
print("\n")

Confusion matrix 
[[394  31   0   1]
 [  1 219   3  59]
 [  0   0 377   0]
 [  5   8   3 337]]




In [7]:
testPred = loaded_model.predict_classes(va_Images[1000].reshape(va_Images[1000].shape[0],1, 244, 244).astype('float32'))
prediction = testPred[0]
print("Class: ",prediction)


Class:  2
